In [1]:
import os.path as op
from scipy.io.idl import readsav
import numpy as np
import warnings
from itertools import islice


In [2]:
filedir = '/Users/bryna/Documents/Physics/HERA/pyuvdata/uvdata/data/fhd_vis_data/'
files = ['1061321792_vis_XX.sav', '1061321792_vis_YY.sav', '1061321792_flags.sav', '1061321792_params.sav',
         '1061321792_settings.txt']
filelist = []
for f in files:
    filelist.append(op.join(filedir, f))
datafiles = {}
params_file = None
flags_file = None
settings_file = None
for file in filelist:
    print(file)
    if file.lower().endswith('_vis_xx.sav'):
        datafiles['xx'] = xx_datafile = file
    elif file.lower().endswith('_vis_yy.sav'):
        datafiles['yy'] = yy_datafile = file
    elif file.lower().endswith('_vis_xy.sav'):
        datafiles['xy'] = xy_datafile = file
    elif file.lower().endswith('_vis_yx.sav'):
        datafiles['yx'] = yx_datafile = file
    elif file.lower().endswith('_params.sav'):
        params_file = file
    elif file.lower().endswith('_flags.sav'):
        flags_file = file
    elif file.lower().endswith('_settings.txt'):
        settings_file = file
    else:
        print(file + ' is not a recognized fhd file type')

if len(datafiles) < 1:
    raise StandardError('No data files included in file list')
if params_file is None:
    raise StandardError('No params file included in file list')
if flags_file is None:
    raise StandardError('No flags file included in file list')
if settings_file is None:
    warnings.warn('No flags file included in file list')

/Users/bryna/Documents/Physics/HERA/pyuvdata/uvdata/data/fhd_vis_data/1061321792_vis_XX.sav
/Users/bryna/Documents/Physics/HERA/pyuvdata/uvdata/data/fhd_vis_data/1061321792_vis_YY.sav
/Users/bryna/Documents/Physics/HERA/pyuvdata/uvdata/data/fhd_vis_data/1061321792_flags.sav
/Users/bryna/Documents/Physics/HERA/pyuvdata/uvdata/data/fhd_vis_data/1061321792_params.sav
/Users/bryna/Documents/Physics/HERA/pyuvdata/uvdata/data/fhd_vis_data/1061321792_settings.txt


In [3]:
vis_data = {}
for pol, file in datafiles.items():
    this_dict = readsav(file, python_dict=True)
    vis_data[pol] = this_dict['vis_ptr']
    this_obs = this_dict['obs']

obs = this_obs
bl_info = obs['BASELINE_INFO'][0]
meta_data = obs['META_DATA'][0]
astrometry = obs['ASTR'][0]
fhd_pol_list = []
for pol in obs['POL_NAMES'][0]:
    fhd_pol_list.append(pol.decode("utf-8").lower())

params_dict = readsav(params_file, python_dict=True)
params = params_dict['params']

flags_dict = readsav(flags_file, python_dict=True)
flag_data = {}
for index, f in enumerate(flags_dict['flag_arr']):
    flag_data[fhd_pol_list[index]] = f

In [4]:
Ntimes = obs['N_TIME'][0]
Nbls = obs['NBASELINES'][0]
Nblts = Ntimes * Nbls
Nfreqs = obs['N_FREQ'][0]
Npols = len(vis_data.keys())
Nspws = 1
spw_array = [1]
vis_units = ['Jy']

lin_pol_order = ['xx','yy','xy','yx']
linear_pol_dict = dict(zip(lin_pol_order, np.arange(5,9)*-1))
pol_list = []
for pol in lin_pol_order:
    if pol in vis_data:
        pol_list.append(linear_pol_dict[pol])
polarization_array = np.asarray(pol_list)


In [5]:
data_array = np.zeros((Nblts,Nspws,Nfreqs,Npols),dtype=np.complex_)
nsample_array = np.zeros((Nblts,Nspws,Nfreqs,Npols),dtype=np.float_)
flag_array = np.zeros((Nblts,Nspws,Nfreqs,Npols),dtype=np.bool_)
for pol,vis in vis_data.items():
    pol_i = pol_list.index(linear_pol_dict[pol])
    data_array[:,0,:, pol_i] = vis
    flag_array[:,0,:, pol_i] = flag_data[pol] <= 0
    nsample_array[:,0,:, pol_i] = np.abs(flag_data[pol])


In [6]:
weights_array = nsample_array * np.where(flag_array, -1, 1)
print(np.min(weights_array), np.max(weights_array))

-0.0 16.048576355


In [6]:
obs.dtype.names

('CODE_VERSION',
 'INSTRUMENT',
 'OBSNAME',
 'DIMENSION',
 'ELEMENTS',
 'NBASELINES',
 'DFT_THRESHOLD',
 'KPIX',
 'DEGPIX',
 'OBSAZ',
 'OBSALT',
 'OBSRA',
 'OBSDEC',
 'ZENRA',
 'ZENDEC',
 'OBSX',
 'OBSY',
 'ZENX',
 'ZENY',
 'PHASERA',
 'PHASEDEC',
 'ORIG_PHASERA',
 'ORIG_PHASEDEC',
 'N_POL',
 'N_TILE',
 'N_TILE_FLAG',
 'N_FREQ',
 'N_FREQ_FLAG',
 'N_TIME',
 'N_TIME_FLAG',
 'N_VIS',
 'N_VIS_IN',
 'N_VIS_RAW',
 'NF_VIS',
 'BEAM_INTEGRAL',
 'POL_NAMES',
 'JD0',
 'MAX_BASELINE',
 'MIN_BASELINE',
 'DELAYS',
 'LON',
 'LAT',
 'ALT',
 'FREQ_CENTER',
 'FREQ_RES',
 'TIME_RES',
 'ASTR',
 'ALPHA',
 'PFLAG',
 'CAL',
 'RESIDUAL',
 'VIS_NOISE',
 'BASELINE_INFO',
 'META_DATA',
 'META_HDR',
 'DEGRID_SPECTRAL_TERMS',
 'GRID_SPECTRAL_TERMS',
 'GRID_INFO',
 'HEALPIX')

In [7]:
meta_data.dtype.names

('INPUT',
 'ANTENNA',
 'TILE',
 'POL',
 'RX',
 'SLOT',
 'FLAG',
 'LENGTH',
 'NORTH',
 'EAST',
 'HEIGHT',
 'GAINS',
 'DELAYS')

In [8]:
astrometry.dtype.names

('NAXIS',
 'CD',
 'CDELT',
 'CRPIX',
 'CRVAL',
 'CTYPE',
 'LONGPOLE',
 'LATPOLE',
 'PV2',
 'PV1',
 'AXES',
 'REVERSE',
 'COORD_SYS',
 'PROJECTION',
 'KNOWN',
 'RADECSYS',
 'EQUINOX',
 'DATEOBS',
 'MJDOBS',
 'X0Y0')

In [7]:

uvw_array = np.zeros((3,Nblts))
uvw_array[0,:] = params['UU'][0]
uvw_array[1,:] = params['VV'][0]
uvw_array[2,:] = params['WW'][0]

# bl_info.JDATE is the only safe date/time to use in FHD files.
# (obs.JD0 and params.TIME are context dependent and are not safe
#   because they depend on the phasing of the visibilities)
# the values in bl_info.JDATE are the JD for each integration.
# We need to expand up to Nblts.
int_times = bl_info['JDATE'][0]
time_array = np.zeros(Nblts)
for ii in range(0, Ntimes):
    time_array[ii*Nbls:(ii+1)*Nbls] = int_times[ii]

ant_1_array = bl_info['TILE_A'][0]
ant_2_array = bl_info['TILE_B'][0]

baseline_array = params['BASELINE_ARR'][0] # should really calculate this from the antenna arrays using bl_to_ij

freq_array = bl_info['FREQ'][0]
phase_center_ra = obs['PHASERA']
phase_center_dec = obs['PHASEDEC']

integration_time = obs['TIME_RES'] # why isn't this quite 2? Do we care?
channel_width = obs['FREQ_RES']

# # --- observation information ---
telescope_name = obs['INSTRUMENT'][0].decode("utf-8")

# a bit of a kludge because nothing like object_name existis in FHD files
# at least for the MWA, obs.ORIG_PHASERA and obs.ORIG_PHASEDEC specify the field the telescope was pointing at
object_name = 'Field RA(deg): ' + str(obs['ORIG_PHASERA'][0]) + ', Dec:' + str(obs['ORIG_PHASEDEC'][0])
if telescope_name.lower() == 'mwa' and np.isclose(obs['ORIG_PHASERA'][0], 0) and np.isclose(obs['ORIG_PHASEDEC'][0], -27):
    object_name = 'EoR 0 Field'

instrument = telescope_name
latitude = obs['LAT'][0]
longitude = obs['LON'][0]
altitude = obs['ALT'][0]
dateobs = min(bl_info['JDATE'][0])

# history
history_list = ['fhd settings info']
if not settings_file is None:
    with open(settings_file) as f:
        head = list(islice(f, 11))
    for line in head:
        newline = ' '.join(str.split(line))
        if not line.startswith('##'):
            history_list.append(newline)
    history = history_list     

phase_center_epoch = astrometry['EQUINOX']

Nants = max([max(ant_1_array), max(ant_2_array)])
antenna_names = bl_info['TILE_NAMES'][0]
antenna_indices = np.arange(Nants)


In [25]:
np.max([len(np.unique(ant_1_array)), len(np.unique(ant_2_array))])

127

In [27]:
print(np.min(uvw_array)*3.0*10**8, np.max(uvw_array)*3.0*10**8)

-2283.18103837 2822.21708403


In [10]:
antenna_names

array([b'      11', b'      12', b'      13', b'      14', b'      15',
       b'      16', b'      17', b'      18', b'      21', b'      22',
       b'      23', b'      24', b'      25', b'      26', b'      27',
       b'      28', b'      31', b'      32', b'      33', b'      34',
       b'      35', b'      36', b'      37', b'      38', b'      41',
       b'      42', b'      43', b'      44', b'      45', b'      46',
       b'      47', b'      48', b'      51', b'      52', b'      53',
       b'      54', b'      55', b'      56', b'      57', b'      58',
       b'      61', b'      62', b'      63', b'      64', b'      65',
       b'      66', b'      67', b'      68', b'      71', b'      72',
       b'      73', b'      74', b'      75', b'      76', b'      77',
       b'      78', b'      81', b'      82', b'      83', b'      84',
       b'      85', b'      86', b'      87', b'      88', b'      91',
       b'      92', b'      93', b'      94', b'      95', b'   

In [11]:
new_antenna_names = []
for i in range(0,len(antenna_names)):
     new_antenna_names.append(antenna_names[i].decode())
    
# new_antenna_names.tolist()
new_antenna_names

['      11',
 '      12',
 '      13',
 '      14',
 '      15',
 '      16',
 '      17',
 '      18',
 '      21',
 '      22',
 '      23',
 '      24',
 '      25',
 '      26',
 '      27',
 '      28',
 '      31',
 '      32',
 '      33',
 '      34',
 '      35',
 '      36',
 '      37',
 '      38',
 '      41',
 '      42',
 '      43',
 '      44',
 '      45',
 '      46',
 '      47',
 '      48',
 '      51',
 '      52',
 '      53',
 '      54',
 '      55',
 '      56',
 '      57',
 '      58',
 '      61',
 '      62',
 '      63',
 '      64',
 '      65',
 '      66',
 '      67',
 '      68',
 '      71',
 '      72',
 '      73',
 '      74',
 '      75',
 '      76',
 '      77',
 '      78',
 '      81',
 '      82',
 '      83',
 '      84',
 '      85',
 '      86',
 '      87',
 '      88',
 '      91',
 '      92',
 '      93',
 '      94',
 '      95',
 '      96',
 '      97',
 '      98',
 '     101',
 '     102',
 '     103',
 '     104',
 '     105',

In [12]:
eastings = 

# # coordinate frame for antenna positions (eg 'ITRF' -also google ECEF)
# # NB: ECEF has x running through long=0 and z through the north pole
# 'xyz_telescope_frame'  : None,
# # coordinates of array center in meters in coordinate frame
# 'x_telescope'  : None,
# 'y_telescope'  : None,
# 'z_telescope'  : None,
# # array giving coordinates of antennas relative to
# # {x,y,z}_telescope in the same frame, (Nants,3)
# 'antenna_positions'  : None,

# # --- other stuff ---
# # the below are copied from AIPS memo 117, but could be revised to
# # merge with other sources of data.
# # when available they are populated. user beware?
# # Greenwich sidereal time at midnight on reference date
# 'GST0'  : None,
# 'RDate'  : None,  # date for which the GST0 or whatever... applies
# # earth's rotation rate in degrees per day
# # (might not be enough sigfigs)
# 'earth_omega'  : 360.985,
# 'DUT1'  : 0.0,        # DUT1 (google it) AIPS 117 calls it UT1UTC
# 'TIMESYS'  : 'UTC',   # We only support UTC


SyntaxError: invalid syntax (<ipython-input-12-04f4253c71dc>, line 1)